# Testing the Code

In [8]:
from Utils import get_features
import numpy as np

In [2]:
vecs = get_features(text='messi.txt',question='What disease was Messi diagnosed with?',num_rel_sentences=3)

In [7]:
list(vecs['sentence 0'].keys())

[his growth hormone treatment,
 Messi,
 Barcelona's greatest-ever youth side,
 Baby Dream Team,
 an integral part]

# Using the Dataset

In [9]:
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import spacy

Getting the Data/Cleaning:

In [11]:
Answer_File = '../../Data/Question_Answer_Dataset_v1.2/S10/question_answer_pairs.txt'

In [12]:
with open(Answer_File,'r',encoding="ISO-8859-1") as f:
    Questions = f.read().split('\n')

In [13]:
Data = {key:[] for key in Questions[0].split('\t')}

In [14]:
keys = list(Data.keys())

In [15]:
for row in range(1,len(Questions)):
    data_point = Questions[row].split('\t')
    if len(data_point)>1:
        for i in range(len(data_point)):
            Data[keys[i]].append(data_point[i])

In [17]:
df = pd.DataFrame(Data)

In [18]:
df.head()

,ArticleTitle,Question,Answer,DifficultyFromQuestioner,DifficultyFromAnswerer,ArticleFile
0,Alessandro_Volta,Was Alessandro Volta a professor of chemistry?,Alessandro Volta was not a professor of chemis...,easy,easy,data/set4/a10
1,Alessandro_Volta,Was Alessandro Volta a professor of chemistry?,No,easy,hard,data/set4/a10
2,Alessandro_Volta,Did Alessandro Volta invent the remotely opera...,Alessandro Volta did invent the remotely opera...,easy,easy,data/set4/a10
3,Alessandro_Volta,Did Alessandro Volta invent the remotely opera...,Yes,easy,easy,data/set4/a10
4,Alessandro_Volta,Was Alessandro Volta taught in public schools?,Volta was taught in public schools.,easy,easy,data/set4/a10


In [20]:
from collections import defaultdict

Articles = defaultdict(list)

# for i,path in enumerate(df['ArticleFile']):
#     if df['ArticleTitle'][i] not in list(Articles.keys()):
#         Articles[df['ArticleTitle'][i]] = Get_Tokens(path)

## Creating a Custom Dataset for our Articles

In [ ]:
def read_file(path):
    with open('../Data/Question_Answer_Dataset_v1.2/'+'S10/'+file_path+'.txt','r') as g:
        text = g.read()
    return Article

class CustomImageDataset(Dataset):
    def __init__(self, article_file_names,questions,answers):
        self.
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = read_image(img_path)
        label = self.img_labels.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label